<a href="https://colab.research.google.com/github/GwonHyeokMIN/-/blob/main/LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn

In [ ]:
class LSTM(nn.Module):
    def __init__(self,input_size,hidden_size,output_size):
        super(LSTM,self).__init__()
        self.hidden_size = hidden_size
        self.i2h = nn.Linear(input_size,hidden_size)
        self.h2h = nn.Linear(hidden_size*2,hidden_size)
        self.tan = nn.Tanh()
        self.sig = nn.Sigmoid()
        self.h20 = nn.Linear(hidden_size,output_size)
        self.dropout = nn.Dropout(0.1)
        self.softmax = nn.LogSoftmax(dim=-1)
    
    def forward(self,input,prev_hidden,prev_cell):
        hidden = self.i2h(input)
        hidden = self.h2h(torch.cat((hidden,prev_hidden),dim=-1))
        cell1 = self.sig(hidden)*prev_cell
        cell2 = self.sig(hidden)*self.tan(hidden)
        cell = cell1 + cell2
        hidden = self.sig(hidden)*self.tan(cell)
        output = self.h20(hidden)
        output = self.dropout(output)
        output = self.softmax(output)

        return output,hidden,cell
        

    def initHidden(self):
        return torch.zeros(1,self.hidden_size)
    
    def initCell(self):
        return torch.zeros(1,self.hidden_size)

In [ ]:
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
x=torch.Tensor([[1,0],[0,1],[1,0],[0,1]]).to(device)
y=torch.Tensor([[1,0,0,1]]).transpose(0,1).type(torch.LongTensor).to(device)
lstm=LSTM(2,64,2).to(device)

criterion=torch.nn.NLLLoss()
optimizer=torch.optim.Adam(lstm.parameters(),lr=1e-4)

epochs=10000
for i in range(epochs):
    total_loss=0
    hidden=lstm.initHidden().to(device)
    cell=lstm.initCell().to(device)

    for time,time_step in enumerate(x):
        output,hidden,cell=lstm(time_step.unsqueeze(0),hidden,cell)
        loss=criterion(output,y[time])
        total_loss+=loss
    optimizer.zero_grad()
    total_loss.backward(retain_graph=True)
    optimizer.step()

    if i%1000 == 0:
        print(total_loss)

In [ ]:
hidden=lstm.initHidden().to(device)
cell=lstm.initCell().to(device)
start=torch.Tensor([[1,0]]).to(device)
for time,_ in enumerate(x):
    output,hidden,cell=lstm(start,hidden,cell)
    value,pred=torch.max(output,dim=-1)
    print(start)
    if pred.item() == 0:
        start=torch.Tensor([[1,0]]).to(device)
    else:
        start=torch.Tensor([[0,1]]).to(device)       
    print(pred)